In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor, VotingRegressor
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
from sklearn.linear_model import LinearRegression, Ridge, Lasso


In [3]:
sample_solution = pd.read_csv("133e814757ed11f0/dataset/sample_solution.csv", index_col='ID')

In [4]:
sample_solution.columns

Index(['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],
      dtype='object')

In [5]:
sample_solution

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,-0.117370,0.348090,0.473673,0.079501,-0.411504,0.015352,0.454957,0.065651,-0.146684,-0.140500
2,-0.503910,-0.250186,-1.412727,-0.523577,-0.577571,-0.294264,-1.396187,-0.856044,-0.003241,-0.246948
3,1.484172,1.272972,1.188539,1.321349,1.472491,1.237584,1.192748,1.575889,0.773926,1.917254
4,0.841616,0.457436,0.534375,0.376650,1.593406,0.157950,0.516430,0.632370,0.376289,-0.446052
5,-0.024147,0.136198,1.174866,-0.197264,2.463520,0.418315,1.185596,0.509797,-0.434762,0.807128
...,...,...,...,...,...,...,...,...,...,...
496,0.022706,-0.047897,0.474691,-0.016551,-0.431127,-0.081999,0.462777,0.089287,-0.608682,0.516307
497,-1.475840,-1.294104,-1.230311,-1.231267,-0.303306,-0.977209,-1.217209,-1.553847,-1.126938,-1.407277
498,1.002035,0.870014,0.037003,0.963333,-0.261270,0.620495,0.029488,0.414339,0.726121,-0.088445


In [21]:
train_dataset = pd.read_csv('133e814757ed11f0/dataset/train.csv')
test_dataset  = pd.read_csv('133e814757ed11f0/dataset/test.csv',index_col='ID')

In [22]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    2000 non-null   float64
 1   Component2_fraction    2000 non-null   float64
 2   Component3_fraction    2000 non-null   float64
 3   Component4_fraction    2000 non-null   float64
 4   Component5_fraction    2000 non-null   float64
 5   Component1_Property1   2000 non-null   float64
 6   Component2_Property1   2000 non-null   float64
 7   Component3_Property1   2000 non-null   float64
 8   Component4_Property1   2000 non-null   float64
 9   Component5_Property1   2000 non-null   float64
 10  Component1_Property2   2000 non-null   float64
 11  Component2_Property2   2000 non-null   float64
 12  Component3_Property2   2000 non-null   float64
 13  Component4_Property2   2000 non-null   float64
 14  Component5_Property2   2000 non-null   float64
 15  Comp

In [23]:
train_dataset, output_blends = train_dataset.iloc[:,:55],train_dataset.iloc[:,55:]

In [24]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    500 non-null    float64
 1   Component2_fraction    500 non-null    float64
 2   Component3_fraction    500 non-null    float64
 3   Component4_fraction    500 non-null    float64
 4   Component5_fraction    500 non-null    float64
 5   Component1_Property1   500 non-null    float64
 6   Component2_Property1   500 non-null    float64
 7   Component3_Property1   500 non-null    float64
 8   Component4_Property1   500 non-null    float64
 9   Component5_Property1   500 non-null    float64
 10  Component1_Property2   500 non-null    float64
 11  Component2_Property2   500 non-null    float64
 12  Component3_Property2   500 non-null    float64
 13  Component4_Property2   500 non-null    float64
 14  Component5_Property2   500 non-null    float64
 15  Component1_

In [25]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.480368,1.044967,-0.450956,0.674572,-0.636394,-1.244963,-1.355050,-0.314423,0.993593,-2.728928
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-1.958826,-0.019603,-0.807923,0.148715,1.439313,-1.160435,-0.014276,-0.135968,-1.221155,0.896222
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,-0.798978,-0.444027,0.148405,-0.793607,0.123834,0.006829,0.668734,0.015449,-0.098661,-0.424314
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-0.534135,1.155513,-0.760428,0.450159,-0.973779,0.052972,-1.024785,0.118951,2.400556,-0.576430
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.389350,1.799238,-0.912374,1.767557,-0.467038,2.104922,0.858593,-0.469110,0.715789,-2.038341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,1.138839,1.666804,-1.413339,0.405253,0.766653,-0.322096,1.399468,1.096369,-0.346225,0.641193
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.782418,0.784366,1.113626,1.328112,-2.537512,0.461525,0.647984,-0.618766,-0.047918,0.397253
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,-0.813747,-0.197880,-0.549162,0.810814,1.567580,-0.694918,-1.710215,-0.233936,-0.133002,-0.284672
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,1.262477,-0.925444,-0.823345,0.427648,-0.161447,0.628131,-0.038484,0.343058,0.448748,0.193507


In [26]:
output_blends

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.489143,0.607589,0.321670,-1.236055,1.601132,1.384662,0.305850,0.193460,0.580374,-0.762738
1,-1.257481,-1.475283,-0.437385,-1.402911,0.147941,-1.143244,-0.439171,-1.379041,-1.280989,-0.503625
2,1.784349,0.450467,0.622687,1.375614,-0.428790,1.161616,0.601289,0.872950,0.660000,2.024576
3,-0.066422,0.483730,-1.865442,-0.046295,-0.163820,-0.209693,-1.840566,0.300293,-0.351336,-1.551914
4,-0.118913,-1.172398,0.301785,-1.787407,-0.493361,-0.528049,0.286344,-0.265192,0.430513,0.735073
...,...,...,...,...,...,...,...,...,...,...
1995,-0.028366,-0.327297,-0.316933,-1.294092,-0.530259,-0.421526,-0.320869,0.709627,-0.737244,-0.744289
1996,-0.449245,0.156778,-0.367445,-0.938615,-0.577451,-0.209996,-0.370505,-0.195531,-0.032834,0.269718
1997,0.029135,0.164890,-0.092942,-1.134490,-0.437479,-0.695636,-0.101073,0.063650,0.624368,-0.477053
1998,-0.232960,-0.464947,0.112536,-0.793522,-0.811272,-1.194914,0.100644,0.760116,-0.751394,-0.857598


In [27]:
pd.DataFrame(output_blends.corr())


,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
BlendProperty1,1.000000,0.726115,0.451207,0.742979,0.154810,0.637866,0.450704,0.644980,0.597190,0.380908
BlendProperty2,0.726115,1.000000,0.431597,0.706317,0.124643,0.638396,0.432767,0.642155,0.603799,0.284212
BlendProperty3,0.451207,0.431597,1.000000,0.432625,0.374290,0.415293,0.997246,0.627322,0.179044,0.512494
BlendProperty4,0.742979,0.706317,0.432625,1.000000,0.153385,0.636376,0.435700,0.641747,0.586330,0.358780
BlendProperty5,0.154810,0.124643,0.374290,0.153385,1.000000,0.152318,0.380571,0.270996,0.028121,0.158801
BlendProperty6,0.637866,0.638396,0.415293,0.636376,0.152318,1.000000,0.416414,0.574911,0.665098,0.255511
BlendProperty7,0.450704,0.432767,0.997246,0.435700,0.380571,0.416414,1.000000,0.629038,0.179592,0.510099
BlendProperty8,0.644980,0.642155,0.627322,0.641747,0.270996,0.574911,0.629038,1.000000,0.474233,0.377589
BlendProperty9,0.597190,0.603799,0.179044,0.586330,0.028121,0.665098,0.179592,0.474233,1.000000,0.158135
BlendProperty10,0.380908,0.284212,0.512494,0.358780,0.158801,0.255511,0.510099,0.377589,0.158135,1.000000


In [28]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# ------------------------------------------------------------------
# 0.  LOAD & COPY ORIGINAL DATAFRAME
# ------------------------------------------------------------------
# assume your raw dataframe is named `df`
# train_dataset = df.copy()

# ------------------------------------------------------------------
# 1.  COLUMN SET‑UP
# ------------------------------------------------------------------
fraction_cols = [f'Component{i}_fraction' for i in range(1, 6)]

property_cols = {}
for prop_idx in range(1, 11):
    property_cols[prop_idx] = [
        f'Component{i}_Property{prop_idx}' for i in range(1, 6)
    ]

# ------------------------------------------------------------------
# 2.  LINEAR BLEND FEATURES
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    train_dataset[w_col] = 0.0
    for comp_idx in range(1, 6):
        train_dataset[w_col] += (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )
    for comp_idx in range(1, 6):
        train_dataset[f'Deviation_{comp_idx}_Prop{prop_idx}'] = (
            train_dataset[f'Component{comp_idx}_Property{prop_idx}']
            - train_dataset[w_col]
        )

# ------------------------------------------------------------------
# 3.  FRACTION‑BASED FEATURES
# ------------------------------------------------------------------
train_dataset['Fraction_Entropy'] = (
    -train_dataset[fraction_cols] * np.log(train_dataset[fraction_cols] + 1e-10)
).sum(axis=1)
train_dataset['Dominant_Fraction'] = train_dataset[fraction_cols].max(axis=1)
train_dataset['Fraction_Range']    = (
    train_dataset[fraction_cols].max(axis=1)
    - train_dataset[fraction_cols].min(axis=1)
)

for i in range(1, 6):
    for j in range(i + 1, 6):
        train_dataset[f'Frac_Interaction_{i}_{j}'] = (
            train_dataset[f'Component{i}_fraction']
            * train_dataset[f'Component{j}_fraction']
        )

# ------------------------------------------------------------------
# 4.  PROPERTY INTERACTION FEATURES
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    train_dataset[f'Prop{prop_idx}_Min']      = train_dataset[cols].min(axis=1)
    train_dataset[f'Prop{prop_idx}_Max']      = train_dataset[cols].max(axis=1)
    train_dataset[f'Prop{prop_idx}_Range']    = (
        train_dataset[f'Prop{prop_idx}_Max'] - train_dataset[f'Prop{prop_idx}_Min']
    )
    train_dataset[f'Prop{prop_idx}_Variance'] = train_dataset[cols].var(axis=1)

    max_vals = train_dataset[cols].max(axis=1)
    for comp_idx in range(1, 6):
        train_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
            train_dataset[cols[comp_idx - 1]] == max_vals
        ).astype(int)

# ------------------------------------------------------------------
# 5.  CROSS‑PROPERTY INTERACTIONS
# ------------------------------------------------------------------
for comp_idx in range(1, 6):
    comp_props = [f'Component{comp_idx}_Property{p}' for p in range(1, 11)]
    train_dataset[f'Comp{comp_idx}_Prop_Mean'] = train_dataset[comp_props].mean(axis=1)
    train_dataset[f'Comp{comp_idx}_Prop_Std']  = train_dataset[comp_props].std(axis=1)

    for prop_idx in range(1, 11):
        train_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Scaled'] = (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )

# ------------------------------------------------------------------
# 6.  NON‑LINEAR TRANSFORMATIONS
# ------------------------------------------------------------------
for comp_idx in range(1, 6):
    frac_col = f'Component{comp_idx}_fraction'
    train_dataset[f'{frac_col}_Sq']   = train_dataset[frac_col] ** 2
    train_dataset[f'{frac_col}_Sqrt'] = np.sqrt(train_dataset[frac_col])

    for prop_idx in range(1, 11):
        col = f'Component{comp_idx}_Property{prop_idx}'
        train_dataset[f'{col}_Log'] = np.log1p(train_dataset[col])
        train_dataset[f'{col}_Sq']  = train_dataset[col] ** 2

# ------------------------------------------------------------------
# 7.  MIXTURE CHARACTERISTICS
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    max_val = train_dataset[cols].max(axis=1)
    min_val = train_dataset[cols].min(axis=1)
    train_dataset[f'Prop{prop_idx}_BlendRatio'] = (
        (max_val - min_val) / (max_val + 1e-10)
    )

    for comp_idx in range(1, 6):
        col = f'Component{comp_idx}_Property{prop_idx}'
        train_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Adj'] = (
            train_dataset[col] * (1 + train_dataset[f'Component{comp_idx}_fraction'])
        )

# ------------------------------------------------------------------
# 8.  CLUSTER LABEL (optional, no target leak)
# ------------------------------------------------------------------
cluster_features = [f'Weighted_Prop{i}' for i in range(1, 11)] + fraction_cols
kmeans = KMeans(n_clusters=5, random_state=42)
# fit on train
_ = kmeans.fit(train_dataset[cluster_features])

# train: compute distances to each center
train_dists = kmeans.transform(train_dataset[cluster_features])
train_dist_df = pd.DataFrame(
    train_dists,
    columns=[f'Cluster_Dist_{i}' for i in range(5)],
    index=train_dataset.index
)
train_dataset = pd.concat([train_dataset, train_dist_df], axis=1)
# ------------------------------------------------------------------
# 9.  TARGET‑INDEPENDENT CONTRIBUTIONS
# ------------------------------------------------------------------
for target_idx in range(1, 11):
    w_col = f'Weighted_Prop{target_idx}'
    for comp_idx in range(1, 6):
        contrib_col = f'Comp{comp_idx}_Contrib_Prop{target_idx}'
        train_dataset[contrib_col] = (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{target_idx}']
            / (train_dataset[w_col] + 1e-10)
        )

# ------------------------------------------------------------------
# 10.  CLEAN‑UP
# ------------------------------------------------------------------
train_dataset.drop(columns=['Blend_Cluster'], inplace=True)


/tmp/ipykernel_92608/4276239847.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
/tmp/ipykernel_92608/4276239847.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_dataset[f'Prop{prop_idx}_Min']      = train_dataset[cols].min(axis=1)
/tmp/ipykernel_92608/4276239847.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

KeyError: "['Blend_Cluster'] not found in axis"

In [29]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Comp1_Contrib_Prop9,Comp2_Contrib_Prop9,Comp3_Contrib_Prop9,Comp4_Contrib_Prop9,Comp5_Contrib_Prop9,Comp1_Contrib_Prop10,Comp2_Contrib_Prop10,Comp3_Contrib_Prop10,Comp4_Contrib_Prop10,Comp5_Contrib_Prop10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,26.889721,0.000000,-50.486585,44.953230,-20.356366,0.553231,0.000000,0.279444,-0.525630,0.692954
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,0.299647,0.049479,1.174108,-0.523234,-0.000000,0.037708,0.007654,0.041973,0.912665,-0.000000
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,0.142130,0.078988,-0.059399,0.882339,-0.044057,-0.009046,-0.885818,-0.046045,0.816799,1.124109
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-1.026313,3.730034,-0.000000,0.242188,-1.945908,-0.033191,1.078722,-0.000000,-0.421151,0.375619
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.101491,0.288618,-0.073178,0.886051,-0.000000,0.544546,0.136689,-0.037341,0.356107,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,0.588973,0.206885,-0.000000,0.108984,0.095158,26.030345,-27.143699,-0.000000,14.549770,-12.436416
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.580659,0.949748,0.000000,1.919396,-1.288486,0.271925,0.622911,-0.000000,-0.054979,0.160144
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,-11.982363,-0.460069,-2.979187,9.739846,6.681772,0.559495,0.217411,0.069391,0.087357,0.066346
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,1.181403,-0.277124,-0.000000,0.144067,-0.048345,0.748410,-0.014673,0.000000,0.192484,0.073780


In [30]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# ------------------------------------------------------------
# 0. COPY RAW TEST DATA
# ------------------------------------------------------------
# assume your test dataframe is named `test_df`
# test_dataset = test_df.copy()

# ------------------------------------------------------------
# 1. COLUMN SETUP
# ------------------------------------------------------------
fraction_cols = [f'Component{i}_fraction' for i in range(1, 6)]
property_cols = {
    prop_idx: [f'Component{i}_Property{prop_idx}' for i in range(1, 6)]
    for prop_idx in range(1, 11)
}

# ------------------------------------------------------------
# 2. LINEAR BLEND FEATURES
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    test_dataset[w_col] = 0.0
    for comp_idx in range(1, 6):
        test_dataset[w_col] += (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )
    for comp_idx in range(1, 6):
        test_dataset[f'Deviation_{comp_idx}_Prop{prop_idx}'] = (
            test_dataset[f'Component{comp_idx}_Property{prop_idx}']
            - test_dataset[w_col]
        )

# ------------------------------------------------------------
# 3. FRACTION-BASED FEATURES
# ------------------------------------------------------------
test_dataset['Fraction_Entropy'] = (
    -test_dataset[fraction_cols] * np.log(test_dataset[fraction_cols] + 1e-10)
).sum(axis=1)
test_dataset['Dominant_Fraction'] = test_dataset[fraction_cols].max(axis=1)
test_dataset['Fraction_Range']    = (
    test_dataset[fraction_cols].max(axis=1)
    - test_dataset[fraction_cols].min(axis=1)
)

for i in range(1, 6):
    for j in range(i + 1, 6):
        test_dataset[f'Frac_Interaction_{i}_{j}'] = (
            test_dataset[f'Component{i}_fraction']
            * test_dataset[f'Component{j}_fraction']
        )

# ------------------------------------------------------------
# 4. PROPERTY INTERACTION FEATURES
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    test_dataset[f'Prop{prop_idx}_Min']      = test_dataset[cols].min(axis=1)
    test_dataset[f'Prop{prop_idx}_Max']      = test_dataset[cols].max(axis=1)
    test_dataset[f'Prop{prop_idx}_Range']    = (
        test_dataset[f'Prop{prop_idx}_Max'] - test_dataset[f'Prop{prop_idx}_Min']
    )
    test_dataset[f'Prop{prop_idx}_Variance'] = test_dataset[cols].var(axis=1)

    max_vals = test_dataset[cols].max(axis=1)
    for comp_idx in range(1, 6):
        test_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
            test_dataset[cols[comp_idx - 1]] == max_vals
        ).astype(int)

# ------------------------------------------------------------
# 5. CROSS-PROPERTY INTERACTIONS
# ------------------------------------------------------------
for comp_idx in range(1, 6):
    comp_props = [f'Component{comp_idx}_Property{p}' for p in range(1, 11)]
    test_dataset[f'Comp{comp_idx}_Prop_Mean'] = test_dataset[comp_props].mean(axis=1)
    test_dataset[f'Comp{comp_idx}_Prop_Std']  = test_dataset[comp_props].std(axis=1)

    for prop_idx in range(1, 11):
        test_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Scaled'] = (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )

# ------------------------------------------------------------
# 6. NON-LINEAR TRANSFORMATIONS
# ------------------------------------------------------------
for comp_idx in range(1, 6):
    frac_col = f'Component{comp_idx}_fraction'
    test_dataset[f'{frac_col}_Sq']   = test_dataset[frac_col] ** 2
    test_dataset[f'{frac_col}_Sqrt'] = np.sqrt(test_dataset[frac_col])

    for prop_idx in range(1, 11):
        col = f'Component{comp_idx}_Property{prop_idx}'
        test_dataset[f'{col}_Log'] = np.log1p(test_dataset[col])
        test_dataset[f'{col}_Sq']  = test_dataset[col] ** 2

# ------------------------------------------------------------
# 7. MIXTURE CHARACTERISTICS
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    max_val = test_dataset[cols].max(axis=1)
    min_val = test_dataset[cols].min(axis=1)
    test_dataset[f'Prop{prop_idx}_BlendRatio'] = (
        (max_val - min_val) / (max_val + 1e-10)
    )

    for comp_idx in range(1, 6):
        col = f'Component{comp_idx}_Property{prop_idx}'
        test_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Adj'] = (
            test_dataset[col] * (1 + test_dataset[f'Component{comp_idx}_fraction'])
        )

# ------------------------------------------------------------
# 8. OPTIONAL: CLUSTER LABEL (no target leak)
# ------------------------------------------------------------
cluster_features = [f'Weighted_Prop{i}' for i in range(1, 11)] + fraction_cols

# test: compute distances
test_dists = kmeans.transform(test_dataset[cluster_features])
test_dist_df = pd.DataFrame(
    test_dists,
    columns=[f'Cluster_Dist_{i}' for i in range(5)],
    index=test_dataset.index
)
test_dataset = pd.concat([test_dataset, test_dist_df], axis=1)

# ------------------------------------------------------------
# 9. TARGET-INDEPENDENT CONTRIBUTIONS
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    for comp_idx in range(1, 6):
        contrib_col = f'Comp{comp_idx}_Contrib_Prop{prop_idx}'
        test_dataset[contrib_col] = (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
            / (test_dataset[w_col] + 1e-10)
        )

# ------------------------------------------------------------
# 10. CLEAN-UP
# ------------------------------------------------------------
test_dataset.drop(columns=['Blend_Cluster'], inplace=True)


/tmp/ipykernel_92608/4065723515.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
/tmp/ipykernel_92608/4065723515.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_dataset[f'Prop{prop_idx}_Min']      = test_dataset[cols].min(axis=1)
/tmp/ipykernel_92608/4065723515.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

KeyError: "['Blend_Cluster'] not found in axis"

#  Add More Featurs

In [31]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Comp1_Contrib_Prop9,Comp2_Contrib_Prop9,Comp3_Contrib_Prop9,Comp4_Contrib_Prop9,Comp5_Contrib_Prop9,Comp1_Contrib_Prop10,Comp2_Contrib_Prop10,Comp3_Contrib_Prop10,Comp4_Contrib_Prop10,Comp5_Contrib_Prop10
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,-6.613360,0.854447,1.264097,-8.880794,14.375610,-5.896530,1.596623,-20.281949,19.282980,6.298877
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,0.000000,0.365887,0.000000,0.702329,-0.068216,-0.000000,1.136398,-0.000000,0.271652,-0.408050
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,0.248486,0.000000,-0.325015,1.111306,-0.034776,-0.709326,0.000000,-0.256346,1.838264,0.127407
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,0.516786,-0.000000,0.024645,0.216259,0.242310,2.080386,-0.000000,0.002130,1.322353,-2.404868
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,0.000000,0.000000,1.973529,-0.973529,0.000000,-0.000000,0.000000,-2.617826,3.617826,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,0.685804,0.001340,0.063311,0.202802,0.046744,0.780083,0.009782,0.030665,0.225261,-0.045791
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,0.118870,0.912187,-0.079909,0.088940,-0.040088,0.529741,0.827689,-0.010229,-0.209668,-0.137534
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,0.190230,0.008189,0.112344,0.314207,0.375031,1.473952,0.033349,0.165223,-0.655451,-0.017072


In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

def create_blending_features(df):
    # Create a copy to avoid modifying original data
    df = df.copy()
    
    # Assumptions about property indices (based on petroleum blending domain knowledge):
    # Property1: API Gravity (density measure)
    # Property2: RVP (Reid Vapor Pressure)
    # Property3: RON (Research Octane Number)
    # Property4: Olefins content
    # Property5: Aromatics content
    # Property6: T10 (10% distillation temperature)
    # Property7: T50 (50% distillation temperature)
    # Property8: T90 (90% distillation temperature)
    # Property9: Viscosity
    # Property10: Cetane Index
    
    # 1. Density and Gravity Features
    for i in range(1, 6):
        # Convert API to Specific Gravity (60/60)
        df[f'Component{i}_SG'] = 141.5 / (df[f'Component{i}_Property1'] + 131.5)
        
        # Calculate density blending index
        df[f'Component{i}_Density_Index'] = df[f'Component{i}_SG'] ** (-0.06)
    
    # Blend density index volumetrically
    df['Blend_Density_Index'] = 0
    for i in range(1, 6):
        df['Blend_Density_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Density_Index']
    
    # Convert back to specific gravity
    df['Blend_SG'] = df['Blend_Density_Index'] ** (-1/0.06)
    df['Blend_API'] = (141.5 / df['Blend_SG']) - 131.5
    
    # 2. Sulfur Blending (gravimetric) - assuming Property2 is sulfur-related
    df['Blend_Sulfur'] = 0
    for i in range(1, 6):
        df['Blend_Sulfur'] += (df[f'Component{i}_fraction'] * df[f'Component{i}_SG'] * 
                              df[f'Component{i}_Property2']) / df['Blend_SG']
    
    # 3. RVP Blending (Index Method)
    rvp_exponent = -0.07  # Texaco method exponent
    for i in range(1, 6):
        df[f'Component{i}_RVP_Index'] = df[f'Component{i}_Property2'] ** rvp_exponent
    
    df['Blend_RVP_Index'] = 0
    for i in range(1, 6):
        df['Blend_RVP_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_RVP_Index']
    
    df['Blend_RVP'] = df['Blend_RVP_Index'] ** (1/rvp_exponent)
    
    # 4. Octane Blending (Ethyl RT-70 method)
    for i in range(1, 6):
        # Simplified octane interaction term
        df[f'Component{i}_RON_BlendValue'] = (
            df[f'Component{i}_Property3'] + 
            0.1 * df[f'Component{i}_Property4'] - 
            0.05 * df[f'Component{i}_Property5']
        )
    
    df['Blend_RON'] = 0
    for i in range(1, 6):
        df['Blend_RON'] += df[f'Component{i}_fraction'] * df[f'Component{i}_RON_BlendValue']
    
    # 5. Distillation Blending (Ethyl S-Curve Model)
    distillation_points = {'T10': 6, 'T50': 7, 'T90': 8}
    for point, prop_idx in distillation_points.items():
        for i in range(1, 6):
            # Simplified distillation blending index
            df[f'Component{i}_{point}_Index'] = np.log(df[f'Component{i}_Property{prop_idx}'])
        
        df[f'Blend_{point}_Index'] = 0
        for i in range(1, 6):
            df[f'Blend_{point}_Index'] += (
                df[f'Component{i}_fraction'] * 
                df[f'Component{i}_{point}_Index']
            )
        
        df[f'Blend_{point}'] = np.exp(df[f'Blend_{point}_Index'])
    
    # 6. Cetane Index (ASTM D-976)
    for i in range(1, 6):
        df[f'Component{i}_Cetane_Index'] = (
            420.34 + 0.016 * df[f'Component{i}_Property1']**2 + 
            0.192 * df[f'Component{i}_Property1'] * np.log(df[f'Component{i}_Property7']) + 
            65.01 * (np.log(df[f'Component{i}_Property7']))**2 - 
            0.0001809 * df[f'Component{i}_Property7']**2
        )
    
    df['Blend_Cetane'] = 0
    for i in range(1, 6):
        df['Blend_Cetane'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Cetane_Index']
    
    # 7. Viscosity Blending
    for i in range(1, 6):
        df[f'Component{i}_Visc_Index'] = np.log(np.log(df[f'Component{i}_Property9'] + 0.8))
    
    df['Blend_Visc_Index'] = 0
    for i in range(1, 6):
        df['Blend_Visc_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Visc_Index']
    
    df['Blend_Viscosity'] = np.exp(np.exp(df['Blend_Visc_Index'])) - 0.8
    
    # # 8. Interaction Terms
    # for i in range(1, 6):
    #     for j in range(i+1, 6):
    #         df[f'Frac_Interaction_{i}_{j}'] = (
    #             df[f'Component{i}_fraction'] * 
    #             df[f'Component{j}_fraction']
    #         )
    
    # 9. Polynomial Features
    # poly = PolynomialFeatures(degree=2, include_bias=False)
    # component_features = [f'Component{i}_fraction' for i in range(1, 6)] + \
    #                      [f'Component{i}_Property1' for i in range(1, 6)] + \
    #                      [f'Component{i}_Property2' for i in range(1, 6)]
    
    # poly_features = poly.fit_transform(df[component_features])
    # poly_cols = [f'Poly_{i}' for i in range(poly_features.shape[1])]
    # poly_df = pd.DataFrame(poly_features, columns=poly_cols)
    # df = pd.concat([df, poly_df], axis=1)
    
    # 10. Mixture Complexity Metrics
    fractions = [f'Component{i}_fraction' for i in range(1, 6)]
    df['Fraction_Entropy'] = (-df[fractions] * np.log(df[fractions] + 1e-9)).sum(axis=1)
    df['Dominant_Fraction'] = df[fractions].max(axis=1)
    
    # 11. Final Cleaning
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)
    
    return df

# Apply feature engineering to both datasets
train_dataset = create_blending_features(train_dataset)
test_dataset = create_blending_features(test_dataset)

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/m

In [33]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,686.546409,567.997235,504.410185,-1.397769,-0.490269,-1.013895,-0.945803,-0.975395,-1.117089,0.681842
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,485.796592,701.435666,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.215462,-1.117089,0.681842
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.975395,-1.117089,0.681842
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,420.665290,534.327773,504.410185,-0.979237,-0.399504,-1.013895,-1.499369,-0.975395,-1.117089,0.681842
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,519.163309,450.234034,504.410185,-0.979237,-0.045815,-1.013895,-0.058737,-0.975395,-1.117089,0.681842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,423.640833,421.011697,504.410185,-0.412355,-0.102118,-1.013895,-1.678307,-0.800862,-1.117089,0.681842
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,519.163309,534.327773,504.410185,-0.979237,-0.776129,-0.432323,-0.280726,-0.975395,-1.117089,0.681842
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-0.740785,-0.148653,-1.117089,0.681842
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,446.859486,430.098069,427.511045,-0.323091,-0.955023,-1.013895,-1.584259,-0.975395,-1.117089,0.681842


In [34]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.299825,-1.303419,0.549224
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,430.534316,423.240164,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-1.219901,-1.303419,0.549224
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,533.739728,663.815825,476.367101,-1.055064,-0.845686,-0.663087,-1.111906,-0.970853,-1.303419,0.549224
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,533.739728,517.803098,476.367101,-1.055064,-2.890751,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,456.068496,424.246233,476.367101,-1.055064,-0.845686,-1.197669,-3.376006,-0.970853,-1.303419,0.549224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,422.428886,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,436.733566,517.803098,476.367101,-1.055064,-0.845686,0.012447,-1.111906,-1.866185,-1.303419,0.549224
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224


In [35]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,686.546409,567.997235,504.410185,-1.397769,-0.490269,-1.013895,-0.945803,-0.975395,-1.117089,0.681842
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,485.796592,701.435666,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.215462,-1.117089,0.681842
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.975395,-1.117089,0.681842
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,420.665290,534.327773,504.410185,-0.979237,-0.399504,-1.013895,-1.499369,-0.975395,-1.117089,0.681842
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,519.163309,450.234034,504.410185,-0.979237,-0.045815,-1.013895,-0.058737,-0.975395,-1.117089,0.681842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,423.640833,421.011697,504.410185,-0.412355,-0.102118,-1.013895,-1.678307,-0.800862,-1.117089,0.681842
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,519.163309,534.327773,504.410185,-0.979237,-0.776129,-0.432323,-0.280726,-0.975395,-1.117089,0.681842
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-0.740785,-0.148653,-1.117089,0.681842
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,446.859486,430.098069,427.511045,-0.323091,-0.955023,-1.013895,-1.584259,-0.975395,-1.117089,0.681842


In [36]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.299825,-1.303419,0.549224
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,430.534316,423.240164,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-1.219901,-1.303419,0.549224
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,533.739728,663.815825,476.367101,-1.055064,-0.845686,-0.663087,-1.111906,-0.970853,-1.303419,0.549224
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,533.739728,517.803098,476.367101,-1.055064,-2.890751,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,456.068496,424.246233,476.367101,-1.055064,-0.845686,-1.197669,-3.376006,-0.970853,-1.303419,0.549224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,422.428886,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,436.733566,517.803098,476.367101,-1.055064,-0.845686,0.012447,-1.111906,-1.866185,-1.303419,0.549224
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224


In [37]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Columns: 564 entries, Component1_fraction to Blend_Viscosity
dtypes: float64(514), int64(50)
memory usage: 2.2 MB


# polynomial _Features creation

In [38]:
train_dataset.corr()

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
Component1_fraction,1.000000,-0.452466,-0.441056,-0.128280,0.028830,-0.015530,-0.032505,-0.000743,0.006344,0.042178,...,0.010787,-0.001266,0.025797,-0.004061,0.011581,0.006799,-0.016042,-0.038140,-0.017628,-0.014670
Component2_fraction,-0.452466,1.000000,-0.430003,-0.162097,0.056293,0.024858,0.020551,-0.039978,-0.007805,-0.025751,...,0.027715,0.013182,-0.014697,-0.000662,-0.017036,0.000670,0.015175,0.008998,0.005702,-0.004778
Component3_fraction,-0.441056,-0.430003,1.000000,-0.205391,0.063267,-0.024512,0.010782,0.030103,-0.001353,-0.005236,...,-0.041647,0.006413,0.001988,-0.004948,0.008847,-0.012119,-0.005723,0.020535,-0.005795,0.006216
Component4_fraction,-0.128280,-0.162097,-0.205391,1.000000,-0.741647,0.017562,0.003803,0.014451,0.005666,-0.008616,...,-0.008742,-0.022524,-0.004275,0.015597,-0.009794,-0.001066,0.002305,0.004947,0.014911,0.012088
Component5_fraction,0.028830,0.056293,0.063267,-0.741647,1.000000,0.000784,-0.004848,-0.004726,-0.004142,-0.007248,...,0.023205,0.002005,-0.019091,-0.007568,0.010096,0.011797,0.009477,0.007961,0.010008,0.005447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Component3_Visc_Index,0.006799,0.000670,-0.012119,-0.001066,0.011797,-0.032115,0.005463,-0.032231,0.051967,-0.003773,...,-0.006205,-0.025820,0.000603,-0.018536,-0.013417,1.000000,-0.033565,-0.014322,0.026345,0.016418
Component4_Visc_Index,-0.016042,0.015175,-0.005723,0.002305,0.009477,-0.001411,0.051331,-0.012324,-0.002322,-0.002776,...,-0.007882,0.029241,0.000466,0.001560,0.024949,-0.033565,1.000000,-0.025208,0.145118,0.142020
Component5_Visc_Index,-0.038140,0.008998,0.020535,0.004947,0.007961,0.011962,0.008306,0.013595,0.021193,0.000460,...,-0.009631,0.010931,0.010468,0.017342,-0.003710,-0.014322,-0.025208,1.000000,0.022677,0.032171
Blend_Visc_Index,-0.017628,0.005702,-0.005795,0.014911,0.010008,-0.033564,-0.007880,0.000299,-0.012339,-0.038177,...,0.006121,0.096620,0.027934,0.058577,0.085858,0.026345,0.145118,0.022677,1.000000,0.910826


In [39]:
test_dataset.corr()

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
Component1_fraction,1.000000,-0.417048,-0.471631,-0.167389,-0.008025,-0.027875,0.053371,-0.033673,0.021528,-0.036265,...,0.087875,-0.015246,8.067531e-03,-0.022745,-0.051726,-0.027253,0.016165,-0.031384,0.051916,0.053481
Component2_fraction,-0.417048,1.000000,-0.414109,-0.232240,0.127492,-0.066247,-0.036146,-0.030215,-0.055298,0.030308,...,-0.021001,-0.068870,-1.798708e-02,0.025552,-0.027303,-0.047138,0.002990,-0.012047,-0.000568,-0.008005
Component3_fraction,-0.471631,-0.414109,1.000000,-0.127871,0.002758,0.022666,-0.008772,0.052068,-0.016428,0.013606,...,-0.057357,0.089360,4.088864e-02,0.001234,0.038046,0.071599,-0.079797,0.058767,-0.037559,-0.028843
Component4_fraction,-0.167389,-0.232240,-0.127871,1.000000,-0.693851,0.082520,-0.042324,0.017643,0.066503,-0.013267,...,-0.044324,0.005887,-2.999864e-02,0.015851,0.079788,0.006494,0.043194,-0.010713,-0.024702,-0.023615
Component5_fraction,-0.008025,0.127492,0.002758,-0.693851,1.000000,0.000474,0.053983,-0.006125,-0.017857,0.011192,...,0.053872,-0.023382,-1.216407e-02,-0.034740,-0.055724,-0.006029,0.048707,-0.011495,0.012440,0.004121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Component3_Visc_Index,-0.027253,-0.047138,0.071599,0.006494,-0.006029,0.041829,0.072967,0.013896,0.062971,-0.039238,...,-0.012524,0.002293,1.024779e-02,0.059567,-0.031880,1.000000,-0.067017,-0.062365,-0.008237,-0.007775
Component4_Visc_Index,0.016165,0.002990,-0.079797,0.043194,0.048707,-0.038648,0.077484,-0.010318,-0.023415,-0.041951,...,0.003464,-0.081022,-1.330528e-02,-0.005346,-0.009518,-0.067017,1.000000,-0.002016,0.224228,0.198419
Component5_Visc_Index,-0.031384,-0.012047,0.058767,-0.010713,-0.011495,0.007695,0.027681,-0.027988,-0.015058,-0.003946,...,-0.219701,0.069831,-1.696028e-02,0.063523,0.047106,-0.062365,-0.002016,1.000000,-0.065482,-0.053600
Blend_Visc_Index,0.051916,-0.000568,-0.037559,-0.024702,0.012440,-0.080831,-0.028490,-0.043929,-0.027459,0.006363,...,0.003366,0.001392,-1.893496e-16,-0.041048,-0.013691,-0.008237,0.224228,-0.065482,1.000000,0.981551


In [40]:
print(list(train_dataset.isna().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [41]:
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# train_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# poly = PolynomialFeatures(degree=(2),interaction_only=True,include_bias=False)
# poly.fit(train_dataset)

In [42]:
# from sklearn.preprocessing import StandardScaler

In [43]:
# train_dataset = poly.transform(train_dataset)
# scaler = StandardScaler().fit(train_dataset)
# train_dataset = scaler.transform(train_dataset)

In [44]:
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset = poly.transform(test_dataset)
# test_dataset = scaler.transform(test_dataset)

In [45]:
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')
# train_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')
# poly = PolynomialFeatures(degree=(2),include_bias=False)
# poly.fit(train_dataset.iloc[:,:55])
# train_log = np.log1p(np.maximum(train_dataset.iloc[:, :55], 1e-6))  

# train_dataset_55 = poly.transform(train_dataset.iloc[:,:55])
# train_dataset = np.hstack([train_dataset_55, train_log,  np.array(train_dataset.iloc[:,55:])])
# scaler = StandardScaler().fit(train_dataset)
# train_dataset = pd.DataFrame(scaler.transform(train_dataset))

In [46]:
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_log = np.log1p(np.maximum(test_dataset.iloc[:, :55], 1e-6))

# test_dataset_55 = poly.transform(test_dataset.iloc[:,:55])
# test_dataset = pd.DataFrame(np.hstack([test_dataset_55, test_log, np.array(test_dataset.iloc[:,55:])]))
# test_dataset = pd.DataFrame(scaler.transform(test_dataset))

In [47]:
# from sklearn.preprocessing import StandardScaler

# # Fix infinities and NaNs
# train_dataset = train_dataset.replace([np.inf, -np.inf], np.nan)
# train_dataset = train_dataset.interpolate(method='linear', limit_direction='both')
# train_dataset = train_dataset.fillna(train_dataset.mean())

# test_dataset = test_dataset.replace([np.inf, -np.inf], np.nan)
# test_dataset = test_dataset.interpolate(method='linear', limit_direction='both')
# test_dataset = test_dataset.fillna(test_dataset.mean())

# # Apply log transform only to relevant features
# log_cols = [col for col in train_dataset.columns if 'fraction' in col or 'Property' in col]
# for col in log_cols:
#     train_dataset[col] = np.log1p(np.maximum(train_dataset[col], 1e-6))
#     test_dataset[col]  = np.log1p(np.maximum(test_dataset[col], 1e-6))

# # Normalize everything
# scaler = StandardScaler()
# train_dataset = pd.DataFrame(scaler.fit_transform(train_dataset), columns=train_dataset.columns)
# test_dataset  = pd.DataFrame(scaler.transform(test_dataset), columns=test_dataset.columns)


In [48]:
from sklearn.feature_selection import VarianceThreshold

# --- 1. Drop near-zero variance features ---
# Threshold is the variance; tune as needed (e.g. 1e-5)
var_selector = VarianceThreshold(threshold=1e-5)
var_selector.fit(train_dataset)

# Keep the mask of good features
mask = var_selector.get_support()
selected_features = train_dataset.columns[mask]

# Reduce both datasets to those features
train_dataset = train_dataset[selected_features]
test_dataset  = test_dataset[selected_features]

# --- 2. Drop highly correlated features ---
# Compute correlation matrix on train only
corr_matrix = train_dataset.corr().abs()
upper_tri  = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Identify columns to drop (corr > 0.98)
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.98)]

# Drop them
train_dataset = train_dataset.drop(columns=to_drop)
test_dataset  = test_dataset.drop(columns=to_drop)

print(f"Dropped {len(to_drop)} correlated features: {to_drop}")
print(f"Final feature count: {train_dataset.shape[1]}")


Dropped 75 correlated features: ['Component4_fraction_Sq', 'Component4_fraction_Sqrt', 'Comp1_Prop1_Adj', 'Comp2_Prop1_Adj', 'Comp3_Prop1_Adj', 'Comp4_Prop1_Adj', 'Comp5_Prop1_Adj', 'Comp1_Prop2_Adj', 'Comp2_Prop2_Adj', 'Comp3_Prop2_Adj', 'Comp4_Prop2_Adj', 'Comp5_Prop2_Adj', 'Comp1_Prop3_Adj', 'Comp2_Prop3_Adj', 'Comp3_Prop3_Adj', 'Comp4_Prop3_Adj', 'Comp5_Prop3_Adj', 'Comp1_Prop4_Adj', 'Comp2_Prop4_Adj', 'Comp3_Prop4_Adj', 'Comp4_Prop4_Adj', 'Comp5_Prop4_Adj', 'Comp1_Prop5_Adj', 'Comp2_Prop5_Adj', 'Comp3_Prop5_Adj', 'Comp4_Prop5_Adj', 'Comp5_Prop5_Adj', 'Comp1_Prop6_Adj', 'Comp2_Prop6_Adj', 'Comp3_Prop6_Adj', 'Comp4_Prop6_Adj', 'Comp5_Prop6_Adj', 'Comp1_Prop7_Adj', 'Comp2_Prop7_Adj', 'Comp3_Prop7_Adj', 'Comp4_Prop7_Adj', 'Comp5_Prop7_Adj', 'Comp1_Prop8_Adj', 'Comp2_Prop8_Adj', 'Comp3_Prop8_Adj', 'Comp4_Prop8_Adj', 'Comp5_Prop8_Adj', 'Comp1_Prop9_Adj', 'Comp2_Prop9_Adj', 'Comp3_Prop9_Adj', 'Comp4_Prop9_Adj', 'Comp5_Prop9_Adj', 'Comp1_Prop10_Adj', 'Comp2_Prop10_Adj', 'Comp3_Prop10_Adj'

In [49]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.299825,-1.303419,0.549224
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,430.534316,423.240164,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-1.219901,-1.303419,0.549224
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,533.739728,663.815825,476.367101,-1.055064,-0.845686,-0.663087,-1.111906,-0.970853,-1.303419,0.549224
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,533.739728,517.803098,476.367101,-1.055064,-2.890751,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,456.068496,424.246233,476.367101,-1.055064,-0.845686,-1.197669,-3.376006,-0.970853,-1.303419,0.549224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,422.428886,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,436.733566,517.803098,476.367101,-1.055064,-0.845686,0.012447,-1.111906,-1.866185,-1.303419,0.549224
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224


In [50]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 483 entries, Component1_fraction to Blend_Viscosity
dtypes: float64(433), int64(50)
memory usage: 7.4 MB


## Splitting the  test data and make testset and validationset

In [51]:
train_X, val_X, train_y, val_y = train_test_split(train_dataset,output_blends, random_state=42, test_size=0.05,)

In [52]:
# sns.pairplot(train_dataset)
print("Train Shape:", train_dataset.shape)
print("Test Shape :", test_dataset.shape)
print("Any NaNs:", train_dataset.isna().sum().sum(), test_dataset.isna().sum().sum())
print("Any Infs:", np.isinf(train_dataset.to_numpy()).sum(), np.isinf(test_dataset.to_numpy()).sum())


Train Shape: (2000, 483)
Test Shape : (500, 483)
Any NaNs: 0 0
Any Infs: 0 0


# Use Tablenet The transformet approach

In [38]:
!pip install tabpfn

In [35]:
from tabpfn import TabPFNClassifier, TabPFNRegressor


In [36]:
tab_reg = TabPFNRegressor(device="auto",random_state=42,n_jobs=12)
multioutput_tab_reg = MultiOutputRegressor(estimator=tab_reg)

In [136]:
multioutput_tab_reg.fit(train_X, train_y)

MultiOutputRegressor(estimator=TabPFNRegressor(n_jobs=12, random_state=42))

In [138]:
# import joblib
# joblib.dump(multioutput_tab_reg, 'multioutput_tabpfn_model.pkl')


['multioutput_tabpfn_model.pkl']

In [37]:
import joblib
loaded_model = joblib.load('models/multioutput_tabpfn_model.pkl')

In [38]:
val_pred_0  = loaded_model.predict(val_X)

In [39]:
tab_reg_pred = val_pred_0 # loaded_model.predict(val_X)
print("MSE: ", mean_squared_error(val_y, tab_reg_pred))
print("MAE: ", mean_absolute_error(val_y, tab_reg_pred))
print("R2 : ", r2_score(val_y, tab_reg_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, tab_reg_pred))

# MSE:  0.0047077895142138
# MAE:  0.037359997630119324
# R2 :  0.9953439831733704
# MAPE: 0.09279724210500717

MSE:  0.0015583925414830446
MAE:  0.02197359874844551
R2 :  0.9984424710273743
MAPE: 0.08065234124660492


In [40]:
pred_tabpfn_0 = loaded_model.predict(train_X)
train_y_0 = train_y-pred_tabpfn_0

In [67]:
# tab_reg_pred = loaded_model.predict(test_dataset)
# tab_reg_pred = pd.DataFrame(tab_reg_pred,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
#        'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
#        'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
# tab_reg_pred.to_csv('_output_tabpfreg.csv')
# tab_reg_pred

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,0.146256,0.228081,0.724894,0.613146,0.353818,0.705358,0.687902,0.360928,-0.336821,0.347265
2,-0.792640,-0.610718,-1.197447,0.063722,-0.728111,-0.120152,-1.191815,-1.050328,-0.933696,0.030162
3,1.761511,1.128711,1.062112,1.069652,2.471631,1.876843,1.027547,1.999289,0.784179,2.252076
4,-0.457512,0.309946,0.901764,-0.718745,1.878562,-0.453220,0.831742,1.781024,0.877961,-0.963787
5,0.158982,-1.220972,1.099539,0.456752,2.454814,0.276937,1.068825,-0.131095,-0.505778,1.060656
...,...,...,...,...,...,...,...,...,...,...
496,0.173990,-0.857383,1.234391,-0.290468,-0.278197,-0.743100,1.197358,-0.444928,-1.457120,-0.432238
497,-2.176230,-1.322172,-1.020217,-2.378759,-0.627009,-2.445888,-1.020925,-1.907284,-1.337142,-1.347159
498,1.999039,2.208122,0.303988,1.188128,0.007698,0.677275,0.290452,0.999529,0.230334,0.483903


In [41]:
train_y#.iloc[:,:1-1]

""
1872
526
173
720
414
...
1130
1294
860
1459


In [ ]:
# from tabpfn import TabPFNClassifier, TabPFNRegressor
# import joblib

In [ ]:
# # Training the model 
# for _ in range(1,11):
#     f'tab_reg_{_}0'  = TabPFNRegressor(device="auto",random_state=42,n_jobs=12)
#     (f'tab_reg_{_}0').fit(np.hstack([train_X, train_y.iloc[:,:_-1]), trian_y[f'BlendProperty{_}'])
#     (f'pred_{_}0') = (f'tab_reg + {_}0').predict(train_X)
#     f'train_y_diff_{_}1' = train_y[f'BlendProperty{_}'] - (pred+ f'_{_}0')
#     f'tab_reg_{_}1'  = TabPFNRegressor(device="auto",random_state=42,n_jobs=12)
#     f'tab_reg_{_}1'.fit(np.hstack([train_X, train_y.iloc[:,:_-1]), (train_y_diff + f'_{_}1'))
#     joblib.dump((f'tab_reg_{_}0'), f"tab_reg_{_}0.pkl")
#     joblib.dump((f'tab_reg_{_}1'), f"tab_reg_{_}1.pkl")    

In [34]:
# # Loading and making the prediction from the model
# prediction  = []
# for _ in range(1,11):
#     f'tab_reg_{_}0' = joblib.load((f'tab_reg_{_}0'), f"tab_reg_{_}0.pkl")
#     f'tab_reg_{_}1' =  joblib.load((f'tab_reg_{_}1'), f"tab_reg_{_}1.pkl") 
#     prediction.append((f'tab_reg_{_}0').predict(np.hstack([train_X, prediction[:,:_-1])))
#     prediction[:,_-1] += (f'tab_reg_{_}1').predict(np.hstack([train_X, prediction[:,:_-1]))

'abc1'

In [42]:
# tab_reg_pred = prediction # loaded_model.predict(val_X)
# print("MSE: ", mean_squared_error(val_y, tab_reg_pred))
# print("MAE: ", mean_absolute_error(val_y, tab_reg_pred))
# print("R2 : ", r2_score(val_y, tab_reg_pred))
# print("MAPE:", mean_absolute_percentage_error(val_y, tab_reg_pred))

0

In [52]:
from tabpfn import TabPFNRegressor
import joblib
import numpy as np
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)


In [38]:
# Containers for models
models0 = {}
models1 = {}
models2 = {}
models3 = {}

# TRAINING
for k in range(1, 11):
    # 1) Build X_k = [train_X, train_y columns 1..k-1]
    if k == 1:
        Xk = train_X.values
    else:
        Xk = np.hstack([train_X.values, train_y.iloc[:, :k-1].values])
    yk = train_y[f'BlendProperty{k}'].values

    # 2) First pass model (stage 0)
    m0 = TabPFNRegressor(device="auto", random_state=42, n_jobs=12)
    m0.fit(Xk, yk)
    pred0 = m0.predict(Xk)

    # 3) Second pass model on residuals (stage 1)
    resid1 = yk - pred0
    m1 = TabPFNRegressor(device="auto", random_state=42, n_jobs=12)
    m1.fit(Xk, resid1)
    pred1 = m1.predict(Xk)

    # 4) Third pass residual-of-residual (stage 2)
    resid2 = yk - (pred0 + pred1)
    m2 = TabPFNRegressor(device="auto", random_state=42, n_jobs=12)
    m2.fit(Xk, resid2)
    pred2 = m2.predict(Xk)

    # 5) Fourth pass residual-of-residual-of-residual (stage 3)
    resid3 = yk - (pred0 + pred1 + pred2)
    m3 = TabPFNRegressor(device="auto", random_state=42, n_jobs=12)
    m3.fit(Xk, resid3)

    # 6) Store models and save
    models0[k] = m0
    models1[k] = m1
    models2[k] = m2
    models3[k] = m3
    joblib.dump(m0, f"tab_reg_{k}0.pkl")
    joblib.dump(m1, f"tab_reg_{k}1.pkl")
    joblib.dump(m2, f"tab_reg_{k}2.pkl")
    joblib.dump(m3, f"tab_reg_{k}3.pkl")



In [39]:
# PREDICTION / EVALUATION
eval_preds = np.zeros((val_X.shape[0], 10))

for k in range(1, 11):
    # Build features for val
    if k == 1:
        Xk_val = val_X.values
    else:
        Xk_val = np.hstack([val_X.values, eval_preds[:, :k-1]])

    # Load models
    m0 = joblib.load(f"tab_reg_{k}0.pkl")
    m1 = joblib.load(f"tab_reg_{k}1.pkl")
    m2 = joblib.load(f"tab_reg_{k}2.pkl")
    m3 = joblib.load(f"tab_reg_{k}3.pkl")

    # Make predictions and sum stages
    p0 = m0.predict(Xk_val)
    p1 = m1.predict(Xk_val)
    p2 = m2.predict(Xk_val)
    p3 = m3.predict(Xk_val)

    eval_preds[:, k-1] = p0 + p1 + p2 + p3



In [40]:
# Compute metrics
print("MSE:", mean_squared_error(val_y.values, eval_preds))
print("MAE:", mean_absolute_error(val_y.values, eval_preds))
print("R2 :", r2_score(val_y.values, eval_preds))
print("MAPE:", mean_absolute_percentage_error(val_y.values, eval_preds))

MSE: 0.009466852415944728
MAE: 0.05503752282344956
R2 : 0.406522287771252
MAPE: 0.19935973660430076


In [41]:
# PREDICTION / EVALUATION
eval_preds = np.zeros((test_dataset.shape[0], 10))

for k in range(1, 11):
    # Build features for val
    if k == 1:
        Xk_val = test_dataset.values
    else:
        Xk_val = np.hstack([test_dataset.values, eval_preds[:, :k-1]])

    # Load models
    m0 = joblib.load(f"tab_reg_{k}0.pkl")
    m1 = joblib.load(f"tab_reg_{k}1.pkl")
    m2 = joblib.load(f"tab_reg_{k}2.pkl")
    m3 = joblib.load(f"tab_reg_{k}3.pkl")

    # Make predictions and sum stages
    p0 = m0.predict(Xk_val)
    p1 = m1.predict(Xk_val)
    p2 = m2.predict(Xk_val)
    p3 = m3.predict(Xk_val)

    eval_preds[:, k-1] = p0 + p1 + p2 + p3



In [42]:
test_pred_tabpfm = eval_preds
test_pred_tabpfm = pd.DataFrame(test_pred_tabpfm,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_tabpfm.to_csv('combine_output_test_pred_tabpfm_check.csv')
test_pred_tabpfm

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,0.148845,0.262678,0.703906,0.640048,0.350738,0.720537,0.680719,0.356276,-0.354980,0.337484
2,-0.795538,-0.598136,-1.193159,0.035167,-0.728268,-0.124294,-1.181531,-1.072566,-0.925527,0.014585
3,1.765951,1.128263,1.072179,1.075381,2.544187,1.875419,1.043127,2.037348,0.756956,2.227070
4,-0.459227,0.305500,0.881599,-0.719728,1.905452,-0.448641,0.855586,1.696070,0.747031,-0.965298
5,0.142876,-1.217153,1.107155,0.443758,2.397340,0.248083,1.076462,-0.163518,-0.466549,1.048567
...,...,...,...,...,...,...,...,...,...,...
496,0.179043,-0.871055,1.207405,-0.291628,-0.276070,-0.733403,1.174684,-0.490458,-1.485863,-0.481735
497,-2.174039,-1.324562,-1.056241,-2.355623,-0.626858,-2.437788,-1.045869,-1.863388,-1.317421,-1.315383
498,1.990788,2.211923,0.337941,1.182781,0.006488,0.679122,0.322285,0.995825,0.183938,0.470461


# Adaboosting

In [53]:
from sklearn.ensemble import AdaBoostRegressor

In [54]:
ada_reg = MultiOutputRegressor(AdaBoostRegressor(estimator=TabPFNRegressor(n_estimators=8,device='cuda',random_state=42,n_jobs=-1),n_estimators=5,learning_rate=0.01,random_state=42,))

In [ ]:
ada_reg.fit(train_X,train_y)

In [ ]:
import joblib
joblib.dump(ada_reg,'ada_boost_multioutput_tabpfn_model.pkl')

In [ ]:
# import joblib
# ada_reg = joblib.load('ada_boost_multioutput_tabpfn_model.pkl')

In [ ]:
tab_reg_pred = ada_reg.predict(val_X)
print("MSE: ", mean_squared_error(val_y, tab_reg_pred))
print("MAE: ", mean_absolute_error(val_y, tab_reg_pred))
print("R2 : ", r2_score(val_y, tab_reg_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, tab_reg_pred))


In [51]:
pred_tabpfms =  loaded_model.predict(test_dataset) + multioutput_tab_reg_0.predict(test_dataset) + multioutput_tab_reg_1.predict(test_dataset)

In [52]:
test_pred_tabpfm = pred_tabpfms
test_pred_tabpfm = pd.DataFrame(test_pred_tabpfm,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_tabpfm.to_csv('combine_output_test_pred_tabpfm.csv')
test_pred_tabpfm

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,0.144660,0.245549,0.712867,0.610593,0.350618,0.707636,0.676811,0.359906,-0.337583,0.336408
2,-0.790911,-0.595550,-1.194027,0.064809,-0.727866,-0.113469,-1.188349,-1.057020,-0.874985,0.017551
3,1.763701,1.133104,1.073073,1.062013,2.544158,1.872890,1.043529,2.012657,0.794215,2.238994
4,-0.455381,0.310517,0.897070,-0.719686,1.894199,-0.453937,0.825443,1.808139,0.871755,-0.975509
5,0.152203,-1.205750,1.118310,0.454448,2.403900,0.265163,1.087767,-0.122754,-0.523214,1.049241
...,...,...,...,...,...,...,...,...,...,...
496,0.179932,-0.858364,1.214646,-0.287842,-0.276840,-0.737055,1.177578,-0.484711,-1.451500,-0.448714
497,-2.173048,-1.341439,-1.045870,-2.379928,-0.626680,-2.451068,-1.044825,-1.910323,-1.333339,-1.343474
498,1.990859,2.201061,0.304565,1.170098,0.006623,0.672645,0.294335,1.017929,0.249123,0.483797
